<a href="https://colab.research.google.com/github/bernia/ETSINF/blob/master/covid_hackaton_ML_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Covid CXR Hackathon: Classiffiers

## Import libraries and load data

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [2]:
!pip install xlrd==2.0.1

     |████████████████████████████████| 96 kB 4.7 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
drive.mount('/content/drive')
df = pd.read_excel('/content/drive/MyDrive/trainClinData.xls')
df.head()

Mounted at /content/drive


,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,WBC,RBC,CRP,Fibrinogen,Glucose,PCT,LDH,INR,D_dimer,Ox_percentage,PaO2,SaO2,PaCO2,pH,CardiovascularDisease,IschemicHeartDisease,AtrialFibrillation,HeartFailure,Ictus,HighBloodPressure,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death
0,82,P_102.png,A,72.000000,1,1.0,37.3,3.0,1.0,0.0,5.37,4.58,3.96,NaN,98.0,NaN,NaN,0.99,NaN,93.0,63.0,93.0,34.0,7.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,SEVERE,0
1,228,P_131.png,D,35.913889,0,1.0,39.3,3.0,1.0,0.0,5.76,5.24,43.40,651.0,NaN,NaN,387.0,1.12,157.0,94.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,MILD,0
2,229,P_132.png,D,57.266667,0,1.0,37.0,3.0,0.0,0.0,11.48,5.71,64.00,700.0,NaN,NaN,338.0,1.23,601.0,94.0,75.0,96.9,37.0,7.42,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,MILD,0
3,96,P_117.png,A,39.000000,1,1.0,37.2,2.0,1.0,0.0,10.17,3.45,0.75,NaN,80.0,NaN,NaN,1.14,NaN,95.0,77.0,97.0,41.0,7.38,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,MILD,0
4,112,P_16.png,A,44.000000,0,1.0,38.0,3.0,1.0,0.0,6.64,5.55,5.20,NaN,126.0,NaN,NaN,NaN,NaN,93.0,65.0,94.0,37.0,7.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEVERE,0


## Prepare data

In [104]:
# Convert Hospital to Categorical (add new columns)
for hospital in df['Hospital'].unique():
    df[f'Hospital_{hospital}']=[0] * len(df)
    df[f'Hospital_{hospital}'][df.index[df['Hospital']==hospital]] = 1

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Row_number,ImageFile,Hospital,Age,Sex,PositivityAtAdmission,Temp_C,DaysFever,Cough,DifficultyInBreathing,WBC,RBC,CRP,Fibrinogen,Glucose,PCT,LDH,INR,D_dimer,Ox_percentage,PaO2,SaO2,PaCO2,pH,CardiovascularDisease,IschemicHeartDisease,AtrialFibrillation,HeartFailure,Ictus,HighBloodPressure,Diabetes,Dementia,BPCO,Cancer,ChronicKidneyDisease,RespiratoryFailure,Obesity,Position,Prognosis,Death,Hospital_A,Hospital_D,Hospital_E,Hospital_F,Hospital_B,Hospital_C
0,82,P_102.png,A,72.000000,1,1.0,37.3,3.0,1.0,0.0,5.37,4.58,3.96,NaN,98.0,NaN,NaN,0.99,NaN,93.0,63.0,93.0,34.0,7.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,SEVERE,0,1,0,0,0,0,0
1,228,P_131.png,D,35.913889,0,1.0,39.3,3.0,1.0,0.0,5.76,5.24,43.40,651.0,NaN,NaN,387.0,1.12,157.0,94.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,MILD,0,0,1,0,0,0,0
2,229,P_132.png,D,57.266667,0,1.0,37.0,3.0,0.0,0.0,11.48,5.71,64.00,700.0,NaN,NaN,338.0,1.23,601.0,94.0,75.0,96.9,37.0,7.42,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,MILD,0,0,1,0,0,0,0
3,96,P_117.png,A,39.000000,1,1.0,37.2,2.0,1.0,0.0,10.17,3.45,0.75,NaN,80.0,NaN,NaN,1.14,NaN,95.0,77.0,97.0,41.0,7.38,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,MILD,0,1,0,0,0,0,0
4,112,P_16.png,A,44.000000,0,1.0,38.0,3.0,1.0,0.0,6.64,5.55,5.20,NaN,126.0,NaN,NaN,NaN,NaN,93.0,65.0,94.0,37.0,7.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEVERE,0,1,0,0,0,0,0


In [4]:
imputed_df = df.copy()

In [5]:
# Impute the data with the mean of each column
features_to_impute = ['Age', 'PositivityAtAdmission', 'Temp_C', 'DaysFever',
                      'Cough', 'DifficultyInBreathing', 'WBC', 'RBC', 'CRP', 
                      'Fibrinogen', 'Glucose', 'PCT', 'LDH', 'INR', 'D_dimer',
                      'Ox_percentage', 'PaO2', 'SaO2', 'PaCO2', 'pH',
                      'CardiovascularDisease', 'IschemicHeartDisease', 
                      'AtrialFibrillation', 'HeartFailure', 'Ictus',
                      'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO',
                      'Cancer', 'ChronicKidneyDisease', 'RespiratoryFailure',
                      'Obesity', 'Position']

features_to_impute_2 = ['Age', 'PositivityAtAdmission', 'Temp_C',
                      'Cough', 'DifficultyInBreathing', 'WBC', 'RBC', 'CRP', 
                      'CardiovascularDisease', 'IschemicHeartDisease', 
                      'AtrialFibrillation', 'HeartFailure', 'Ictus',
                      'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO',
                      'Cancer', 'ChronicKidneyDisease', 'RespiratoryFailure'
                      ]

for feature in features_to_impute:
    imputed_df[feature] = imputed_df[feature].fillna(imputed_df[feature].mean())

In [9]:
features_to_use = ['Sex','Age', 'PositivityAtAdmission', 'Temp_C', 'DaysFever',
                      'Cough', 'DifficultyInBreathing', 'WBC', 'RBC', 'CRP', 
                      'Fibrinogen', 'Glucose', 'PCT', 'LDH', 'INR', 'D_dimer',
                      'Ox_percentage', 'PaO2', 'SaO2', 'PaCO2', 'pH',
                      'CardiovascularDisease', 'IschemicHeartDisease', 
                      'AtrialFibrillation', 'HeartFailure', 'Ictus',
                      'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO',
                      'Cancer', 'ChronicKidneyDisease', 'RespiratoryFailure',
                      'Obesity', 'Position'] #, 'Hospital_A', 'Hospital_B',
                      #'Hospital_C', 'Hospital_D', 'Hospital_E', 'Hospital_F']

features_to_use_2 = ['Sex','Age', 'PositivityAtAdmission', 'Temp_C', 'DaysFever',
                      'Cough', 'DifficultyInBreathing', 'WBC', 'RBC', 'CRP',
                      'CardiovascularDisease', 'IschemicHeartDisease', 
                      'AtrialFibrillation', 'HeartFailure', 'Ictus',
                      'HighBloodPressure', 'Diabetes', 'Dementia', 'BPCO',
                      'Cancer', 'ChronicKidneyDisease', 'RespiratoryFailure']

target_feature = 'Prognosis'

In [7]:
# Z score normalization on Non Binary variables
non_binary_variables = ['Age', 'Temp_C', 'DaysFever', 'WBC', 'RBC', 'CRP',
                        'Fibrinogen', 'Glucose', 'PCT', 'LDH', 'INR', 'D_dimer',
                        'Ox_percentage', 'PaO2', 'SaO2','PaCO2', 'pH']

# Extract and normalize them
#non_binary_values = imputed_df[non_binary_variables].to_numpy(copy=True)
#scaler = preprocessing.StandardScaler().fit(non_binary_values)
#non_binary_values = scaler.transform(non_binary_values)

for var in non_binary_variables:
  imputed_df[var] = (imputed_df[var] - imputed_df[var].mean()) / imputed_df[var].std()


In [10]:
X = imputed_df[features_to_use].to_numpy(copy=True)
Y = imputed_df[target_feature].to_numpy(copy=True)

In [11]:
print(X.shape, Y.shape)

(1103, 35) (1103,)


### XGBoost Classifier

In [145]:
xgb_classifier = XGBClassifier(n_estimators=100,
                               objective='binary:logistic') 
results = cross_val_score(xgb_classifier, X, Y, scoring='accuracy')
print(results)
print(f"Average accuraccy accross all the folds: {results.mean()}")

[0.64705882 0.66968326 0.66968326 0.64545455 0.73181818]
Average accuraccy accross all the folds: 0.6727396133278487


### Other Scikit learn classifiers

In [146]:
model_1 = GradientBoostingClassifier(loss="deviance", learning_rate=0.1, n_estimators=200, verbose=0)

results_1 = cross_val_score(model_1, X, Y, scoring='accuracy')
print(f"Gradient Boosting Results: {results_1.mean()}")

Gradient Boosting Results: 0.6528177704648293


In [147]:
model_2 = svm.LinearSVC(C=1000, tol=0.1, loss='hinge', max_iter=5000)

results_2 = cross_val_score(model_2, X, Y, scoring='accuracy')
print(f"Linear SVM Results: {results_2.mean()}")

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Linear SVM Results: 0.6074454956807898


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [148]:
model_3 = svm.SVC()

results_3 = cross_val_score(model_3, X, Y, scoring='accuracy')
print(f"SVM with RBF kernel Results: {results_3.mean()}")

SVM with RBF kernel Results: 0.6518716577540108


In [149]:
model_4 = SGDClassifier(alpha=1e-5, loss='squared_hinge', n_jobs=-1, tol=0.001)

results_4 = cross_val_score(model_4, X, Y, scoring='accuracy')
print(f"SGD Classifier Results: {results_4.mean()}")

SGD Classifier Results: 0.5838420403126285


In [150]:
model_5 = neighbors.KNeighborsClassifier(n_neighbors=20)

results_5 = cross_val_score(model_5, X, Y, scoring='accuracy')
print(f"K nearest neighbours Results: {results_5.mean()}")

K nearest neighbours Results: 0.6418881118881119


In [153]:
model_6 = RandomForestClassifier(n_estimators=1000)

results_6 = cross_val_score(model_6, X, Y, scoring='accuracy')
print(f"Random Forest Results: {results_6.mean()}")

Random Forest Results: 0.6799794323323736


In [154]:
model_7 = LogisticRegression(max_iter=1000)

results_7 = cross_val_score(model_7, X, Y, scoring='accuracy')
print(f"Logistic Regression Results: {results_7.mean()}")

Logistic Regression Results: 0.6564500205676677


In [155]:
model_8 = DecisionTreeClassifier()

results_8 = cross_val_score(model_8, X, Y, scoring='accuracy')
print(f"Decision Tree Results: {results_8.mean()}")

Decision Tree Results: 0.5838584944467298


In [152]:
model_9 = MLPClassifier(hidden_layer_sizes=(32, 16),
                                activation="relu",
                                solver="adam",
                                alpha=0.0001, 
                                batch_size=100, 
                                learning_rate_init=0.001,
                                max_iter=20000,
                                shuffle=True,
                                verbose=False, 
                                warm_start=False)

results_9 = cross_val_score(model_9, X, Y, scoring='accuracy')
print(f" Results: {results_9.mean()}")

 Results: 0.6146976552858906


In [126]:
print(f"Gradient Boosting Results: {results_1.mean()}")
print(f"Linear SVM Results: {results_2.mean()}")
print(f"SVM with RBF kernel Results: {results_3.mean()}")
print(f"SGD Classifier Results: {results_4.mean()}")
print(f"K nearest neighbours Results: {results_5.mean()}")
print(f"Random Forest Results: {results_6.mean()}")
print(f"Logistic Regression Results: {results_7.mean()}")
print(f"Decision Tree Results: {results_8.mean()}")
print(f"MLP Results: {results_9.mean()}")

Gradient Boosting Results: 0.7135129576306047
Linear SVM Results: 0.5802550390785685
SVM with RBF kernel Results: 0.6092060880296175
SGD Classifier Results: 0.5648004936240231
K nearest neighbours Results: 0.6636528177704648
Random Forest Results: 0.7343562320032909
Logistic Regression Results: 0.7370876182640889
Decision Tree Results: 0.6291772932949403
MLP Results: 0.6708021390374331


### Grid search Cross Validation

In [48]:
# Gradient Boosting Classifier
model_1 = GradientBoostingClassifier()
params_1 = {
    "loss": ["deviance", "exponential"],
    "learning_rate": [0.1, 0.01, 0.001, 0.0001],
    "n_estimators": [150, 250, 350, 450, 550],
}


# Linear SVC
model_2 = svm.LinearSVC()
params_2 = {
    "loss": ["hinge", "squared_hinge"],
    "C": [1, 10, 100, 1000],
    "tol": [0.01, 0.1, 1, 10, 50],
    "max_iter": [100000],
}


# SVC with RBF kernel
model_3 = svm.SVC()
params_3 = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"]
}

# SGD Classifier
model_4 = SGDClassifier()
params_4 = {
    "loss": ["hinge", "squared_hinge", "perceptron"],
    "alpha": [0.1, 0.01, 0.001, 0.0001],
    "tol": [0.01, 0.1, 10],
    "n_jobs": [-1],
}

# KNN
model_5 = neighbors.KNeighborsClassifier()
params_5 = {"n_neighbors": [10, 20, 30, 40, 50] }


# Random Forest - Should not be estimated with grid search
# First, find best parameters with one Decision Tree and then use them for RF
model_6 = RandomForestClassifier()
params_6 = {
    "criterion": ["gini", "entropy"],
    "n_estimators": [100, 200, 500, 1000]
}

# Logistic Regression
model_7 = LogisticRegression()
params_7 = {
    "penalty": ["l1", "l2", "elasticnet"],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
    "fit_intercept": [True, False],
    "tol": [0.01, 0.001, 0.0001]
}

# Decision Tree
model_8 = DecisionTreeClassifier()
params_8 = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"]
}

# Multi Layer Perceptron
model_9 = MLPClassifier(
                    max_iter=20000,
                    verbose=False,
                    warm_start=False,
                )
params_9 = {
    "hidden_layer_sizes": [
        (32, 16)
    ],
    "solver": ["adam", "sgd"],
    "alpha": [0.00001, 0.0001],
    "activation": ["logistic", "relu"],
    "learning_rate": ["constant"],#, "adaptive"],
    "learning_rate_init": [0.1, 0.01, 0.001, 0.0001, 0.00001],
}


In [14]:
classifier1 = GridSearchCV(model_1, params_1, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier1.fit(X, Y)
print(classifier1.best_score_)
print(classifier1.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
0.7198519127930891
{'learning_rate': 0.1, 'loss': 'deviance', 'n_estimators': 450}


In [23]:
classifier2 = GridSearchCV(model_2, params_2, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier2.fit(X, Y)
print(classifier2.best_score_)
print(classifier2.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
0.7343562320032909
{'C': 10, 'loss': 'hinge', 'max_iter': 100000, 'tol': 0.01}


In [24]:
classifier3 = GridSearchCV(model_3, params_3, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier3.fit(X, Y)
print(classifier3.best_score_)
print(classifier3.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
0.7316289592760181
{'kernel': 'linear'}


In [25]:
classifier4 = GridSearchCV(model_4, params_4, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier4.fit(X, Y)
print(classifier4.best_score_)
print(classifier4.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0.7262320032908267
{'alpha': 0.01, 'loss': 'hinge', 'n_jobs': -1, 'tol': 10}


In [26]:
classifier5 = GridSearchCV(model_5, params_5, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier5.fit(X, Y)
print(classifier5.best_score_)
print(classifier5.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
0.6808350473056355
{'n_neighbors': 20}


In [49]:
classifier6 = GridSearchCV(model_6, params_6, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier6.fit(X, Y)
print(classifier6.best_score_)
print(classifier6.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.7416042780748663
{'criterion': 'entropy', 'n_estimators': 100}


In [37]:
classifier7 = GridSearchCV(model_7, params_7, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier7.fit(X, Y)
print(classifier7.best_score_)
print(classifier7.best_params_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
0.7334512546277252
{'fit_intercept': False, 'penalty': 'l2', 'solver': 'sag', 'tol': 0.01}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solv

In [38]:
classifier8 = GridSearchCV(model_8, params_8, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier8.fit(X, Y)
print(classifier8.best_score_)
print(classifier8.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
0.6509132044426162
{'criterion': 'gini', 'splitter': 'best'}


In [44]:
classifier9 = GridSearchCV(model_9, params_9, cv=5, n_jobs=-1, verbose=1, scoring="accuracy")
classifier9.fit(X, Y)
print(classifier9.best_score_)
print(classifier9.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
0.7352612093788565
{'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (32, 16), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'sgd'}
